In [1]:
import tensorflow as tf
import numpy as np
from tqdm.notebook import tqdm

2021-12-21 16:13:45.492015: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-21 16:13:45.492035: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## MCTS Chess Neural Network
---

[notes go here]

In [2]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

class SimpleChessNet(Model):
    
    def __init__(self, action_size,
                 board_input_shape=(8,8,6),
                 n_conv_channels=12,
                 dropout_rate=0.5):
        
        super(SimpleChessNet, self).__init__()
        
        self.board_x, self.board_y, self.board_channels = board_input_shape
        self.action_size = action_size
        
        self.input_layer = Input(shape=board_input_shape)
        
        h_conv1 = Activation('elu')(BatchNormalization(axis=3)(Conv2D(n_conv_channels, 3, padding='same', use_bias=False)(self.input_layer))) # batch_size  x board_x x board_y x num_channels
        h_conv2 = Activation('elu')(BatchNormalization(axis=3)(Conv2D(n_conv_channels, 3, padding='same', use_bias=False)(h_conv1)))          # batch_size  x board_x x board_y x num_channels
        h_conv3 = Activation('elu')(BatchNormalization(axis=3)(Conv2D(n_conv_channels, 3, padding='valid', use_bias=False)(h_conv2)))         # batch_size  x (board_x-2) x (board_y-2) x num_channels
        h_conv4 = Activation('elu')(BatchNormalization(axis=3)(Conv2D(n_conv_channels, 3, padding='valid', use_bias=False)(h_conv3)))         # batch_size  x (board_x-4) x (board_y-4) x num_channels
        h_conv4_flat = Flatten()(h_conv4)
        
        s_fc1 = Dropout(dropout_rate)(Activation('elu')(BatchNormalization(axis=1)(Dense(1024, use_bias=False)(h_conv4_flat))))  # batch_size x 1024
        s_fc2 = Dropout(dropout_rate)(Activation('elu')(BatchNormalization(axis=1)(Dense(512, use_bias=False)(s_fc1))))          # batch_size x 1024
        
        # get the policy and value estimates:
        self.pi = Dense(self.action_size, activation='softmax', name='pi')(s_fc2) # batch_size x self.action_size
        self.v = Dense(1, activation='tanh', name='v')(s_fc2)
        
        self.model = Model(inputs=self.input_layer, outputs=[self.pi, self.v])
        # self.model.compile(loss=['categorical_crossentropy', 'mean_squared_error'], optimizer=Adam(learning_rate))
        
    def call(self, x):
        return self.model(x)
        
    def __call__(self, x):
        return self.model(x)
        
    def compute_loss(self, x, pi, v):
        policy_pred, v_pred = self.__call__(x)
        v_loss = tf.reduce_mean((v_pred - v)**2)
        pi_loss = tf.reduce_mean(-pi*tf.math.log(policy_pred))
        total_loss = tf.add_n([(v_loss + pi_loss)] + self.losses) 
        return v_loss, pi_loss, total_loss
    
    def save(self, path):
        tf.saved_model.save(self.model, path)


In [3]:
nn = SimpleChessNet(action_size=64*64)
nn.model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8, 8, 6)]    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 8, 8, 12)     648         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 8, 8, 12)     48          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 8, 8, 12)     0           batch_normalization[0][0]        
______________________________________________________________________________________________

2021-12-21 16:13:46.998778: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-21 16:13:46.998813: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-21 16:13:46.998837: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (arcturus): /proc/driver/nvidia/version does not exist
2021-12-21 16:13:46.999134: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
from tensorflow.keras.optimizers import *

class ChessNetModule(tf.Module):
    
    def __init__(self, learning_rate=1e-3):
        super(ChessNetModule, self).__init__()
        self.model = SimpleChessNet(action_size=64*64)
        self.optimizer = Adam(learning_rate)
                
        
        
    @tf.function(input_signature=[tf.TensorSpec([None,8,8,6])])
    def __call__(self, x):
        return self.model(x)
    
    @tf.function(input_signature=[tf.TensorSpec([None,8,8,6], tf.float32), 
                                  tf.TensorSpec([None,64*64], tf.float32), 
                                  tf.TensorSpec([None], tf.float32)])
    def train_step(self, x, pi, v):
        with tf.GradientTape() as tape:
            v_loss, pi_loss, total_loss = self.model.compute_loss(x, pi, v)
        
        grads = tape.gradient(total_loss, self.model.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_weights))
        
        return v_loss, pi_loss, total_loss
    
    def save(self, path):
        tf.saved_model.save(self, path, 
        signatures={
            'chessnet_serve' : 
                self.__call__.get_concrete_function(tf.TensorSpec([None,8,8,6], tf.float32)),
            'chessnet_train' : 
                self.train_step.get_concrete_function(tf.TensorSpec([None,8,8,6], tf.float32),
                                                      tf.TensorSpec([None,64*64], tf.float32),
                                                      tf.TensorSpec([None], tf.float32))
        })

In [5]:
nn_mod = ChessNetModule()
nn_mod.save('simple_chess_net')

2021-12-21 16:13:48.730096: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: simple_chess_net/assets
